In [ ]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Dropout
import os
import shutil
from utils_plot import plot_candlestick_chart
from predictProcessor import PredictProcessor
from inputProcessor import InputProcessor

# Define the path to the data folder
data_folder = 'data/learn'
output_folder = 'training_plots'  # Specify the output folder

# Clean the output folder before each run
if os.path.exists(output_folder):
    shutil.rmtree(output_folder)  # Remove the folder and its contents

# Create the output folder
os.makedirs(output_folder)

# Get a list of all files in the data folder
file_list = [filename for filename in os.listdir(data_folder) if filename.endswith('.csv')]

# Loop through each file
for filename in file_list:
    file_path = os.path.join(data_folder, filename)
    stock_name = os.path.splitext(filename)[0]
    data = InputProcessor(file_path)

    # Load model weights if available
    try:
        model = load_model('a_model')
        print("Model loaded successfully.")
    except:
        print("No model found. Training from scratch.")
        # Build the LSTM model
        model = Sequential()
        model.add(LSTM(64, activation='relu', input_shape=(data.trainX.shape[1], data.trainX.shape[2]), return_sequences=True))
        model.add(LSTM(32, activation='relu', return_sequences=False))
        model.add(Dropout(0.2))
        model.add(Dense(units=5))  # Output layer for 5 classes (Wave 1 through 5)
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

        """ 
        model = Sequential()
        model.add(LSTM(128, activation='relu', input_shape=(
                    data.trainX.shape[1], data.trainX.shape[2]), return_sequences=True))
        model.add(LSTM(64, activation='relu', return_sequences=False))
        model.add(Dropout(0.3))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) """

        """ model = Sequential()
        model.add(LSTM(64, activation='relu', input_shape=(
                    data.trainX.shape[1], data.trainX.shape[2]), return_sequences=True))
        model.add(LSTM(32, activation='relu', return_sequences=False))
        model.add(Dropout(0.2))
        model.add(Dense(units=1, activation='sigmoid'))  # Output layer with 1 units for Wave category
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) """

        """     model = Sequential()
        model.add(LSTM(units=64, activation='relu', input_shape=(data.n_past, 5)))  # Adjust input_shape here
        model.add(Dense(units=32, activation='relu'))
        model.add(Dense(units=5, activation='sigmoid'))  # Output layer with 5 units for Wave1 to Wave5
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) """

    model.summary()

    print("input_shape=(data.trainX.shape[1], data.trainX.shape[2])", data.trainX.shape[1], data.trainX.shape[2])
    print("trainX", data.trainX)
    print("trainY", data.trainY)

    # Train the model
    history = model.fit(data.trainX, data.trainY, epochs=20, batch_size=1, validation_split=0.1, verbose=1)
    # fit the model
    model.save('a_model')


    print('history', history)
    # Save the plot as an image file in the output folder
    #output_filename = os.path.join(output_folder, os.path.splitext(filename)[0] + '_plot.png')
    #print_plot_model_loss(output_filename, history.history['loss'], history.history['val_loss'])

    # Evaluate the model on the test data
    #result = model.evaluate(X_test_sequences, y_test_sequences, verbose=1)
    #print("Evaluation result:", result)
    #loss, accuracy = model.evaluate(X_test_sequences, y_test_sequences, verbose=1)
    #print(f'Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}')

    PredictProcessor(model, data, stock_name, output_folder)

